In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Cargar el dataset
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True
)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.OEZQ46_3.0.1/mnist-train.tfrecord*...:   0%|          | 0…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/mnist/incomplete.OEZQ46_3.0.1/mnist-test.tfrecord*...:   0%|          | 0/…

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [2]:
# Preprocesamiento
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

ds_train = ds_train.map(normalize_img).cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
ds_test = ds_test.map(normalize_img).batch(32).cache().prefetch(tf.data.AUTOTUNE)


In [3]:
# Crear el modelo
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax'),
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [4]:
# Entrenar
model.fit(ds_train, epochs=5, validation_data=ds_test)
test_loss, test_acc = model.evaluate(ds_test)
print("Precisión en test:", test_acc)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.8727 - loss: 0.4318 - val_accuracy: 0.9584 - val_loss: 0.1289
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 12s 4ms/step - accuracy: 0.9644 - loss: 0.1155 - val_accuracy: 0.9697 - val_loss: 0.0976
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9764 - loss: 0.0772 - val_accuracy: 0.9735 - val_loss: 0.0822
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.9831 - loss: 0.0558 - val_accuracy: 0.9758 - val_loss: 0.0832
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.9858 - loss: 0.0453 - val_accuracy: 0.9755 - val_loss: 0.0789
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9734 - loss: 0.0802
Precisión en test: 0.9754999876022339


In [5]:
# Guardar el modelo
model.save('mnist_model.h5',save_format="tf")

# Convertir a TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Guardar como archivo .tflite
with open('mnist_model.tflite', 'wb') as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpz4tzh66n'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  138013417720720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138013417721488: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138013417420368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138013417718992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138013417419792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  138013417720144: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [6]:
print("Versión de TensorFlow:", tf.__version__)
print("Versión de Keras:", tf.keras.__version__)

Versión de TensorFlow: 2.18.0
Versión de Keras: 3.8.0
